# Transforming NBA Box Score Data and General Game Data
In the last post, I went over the data source as well as the methodology for extracting NBA Data. Currently, I'm pulling play-by-play and box score data, along with all the general game data from NBA.com, player transaction data from RealGM.com, and salary data from Sportrac. Here's a quick summary of the raw data, its format, and source, as well as the two data sources that I'm going to work on transforming in this post, the Box Score data and General Game Data:

| Raw Data | Format | Source |
| ----------------------- | ----------- | --------------------- |
| Play-By-Play Data | JSON | NBA.com |
| Play-By-Play Data | HTML | Basketball Reference |
| <span style="font-weight: bold; color: blue;">Box Score Data</span> | <span style="font-weight: bold; color: blue;">JSON</span> | <span style="font-weight: bold; color: blue;">NBA.com</span> |
| <span style="font-weight: bold; color: blue;">General Game Data</span> | <span style="font-weight: bold; color: blue;">JSON</span> | <span style="font-weight: bold; color: blue;">NBA.com</span> |
| Player Transaction Data | HTML | RealGM.com |
| Player Salary Data | HTML | Sportrac.com |

I'm going to transform this data into a relational database, which will likely be changed/altered as time goes on. I'll provide the transformation that I plan to run for each table, and how the transformation of the source data will work. 

## Box Score Table
The box score data can be found nested in the NBA.com JSON file within the 'homeTeam' and 'awayTeam' keys. I have the JSON file saved locally for the purposes of the below code:

In [10]:
import json

# Read the JSON File
f = open('2022-04-10-was-vs-cha-0022101217.json')
json_object = json.load(f)
# Extract the Box Score Data
box_score_home = json_object['props']['pageProps']['game']['homeTeam']
box_score_away = json_object['props']['pageProps']['game']['awayTeam']

We can now take this json data, and turn it into a flat file using the pandas package in python:

In [5]:
import pandas as pd


def generate_box_score(box_score_dict):
    rows = []
    for player in box_score_dict['players']:
        row = {'player_id' : player['personId']}
        player_name = '{} {}'.format(player['firstName'], player['familyName'])
        row.update({'player_name' : player_name}) # add the player name
        row.update({'status' : player['comment']}) # player's status
        row.update(player['statistics']) # add the player stats
        rows.append(row)
    return pd.DataFrame(rows)

The resulting dataframe looks as follows:

In [7]:
generate_box_score(box_score_home)

,player_id,player_name,status,minutes,fieldGoalsMade,fieldGoalsAttempted,fieldGoalsPercentage,threePointersMade,threePointersAttempted,threePointersPercentage,freeThrowsMade,freeThrowsAttempted,freeThrowsPercentage,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints
0,1628970,Miles Bridges,,27:49,3,12,0.250,1,4,0.250,2,2,1,1,6,7,5,0,1,0,1,9,0
1,1629023,P.J. Washington,,24:33,4,4,1.000,0,0,0.000,1,1,1,0,1,1,1,0,1,1,4,9,8
2,203486,Mason Plumlee,,21:08,4,6,0.667,0,0,0.000,0,2,0,1,7,8,2,0,2,2,3,8,-9
3,1626179,Terry Rozier,,31:37,10,18,0.556,5,9,0.556,0,0,0,0,3,3,6,0,0,1,2,25,6
4,1630163,LaMelo Ball,,28:31,8,16,0.500,6,7,0.857,2,2,1,2,8,10,9,1,0,8,1,24,19
5,1626149,Montrezl Harrell,,22:27,4,6,0.667,0,0,0.000,0,0,0,2,0,2,4,1,1,2,1,8,24
6,1628998,Cody Martin,,24:03,1,3,0.333,0,1,0.000,3,3,1,3,2,5,3,2,1,1,1,5,13
7,1626162,Kelly Oubre Jr.,,16:25,2,5,0.400,2,5,0.400,0,2,0,0,3,3,1,1,0,0,2,6,19
8,1629667,Jalen McDaniels,,20:36,6,8,0.750,2,2,1.000,0,0,0,2,4,6,2,0,1,1,0,14,17
9,202738,Isaiah Thomas,,12:31,5,8,0.625,2,4,0.500,2,2,1,0,1,1,1,1,0,0,2,14,-1


#### Note on Active/Inactive/DNP Players
One additional piece of information I want to capture that isn't always available in the box score data is whether or not the player is active, and if they're not, the reason for their absence. 

If a player is "DNP" (did not play) it can be found in the box score, however, inactive players don't appear there. In addition, older box scores sometimes don't even list "DNP" players (it's unclear when exactly teams started including them, but it appears to be sometime in the mid 90s), and they definitely don't provide any information on whether or not a player was "inactive". For my purposes, it really doesn't matter if a player is "inactive" or "DNP", either shows that the player was on the roster and didn't play in the game, which is all I want. 

Capturing this with more recent data is fairly easy because there is a nested dictionary specifically for inactive players that is populated in the data (I think it's post-2005-ish when it begins to consistently appear), however, I haven't assessed its reliability. 

The only real method for ensuring that the data is accurate would be more complicated than what I want do do here, and would require tracking a player's start and end dates with every team, and then merging those dates with each team's game logs. This would be made even more complicated by the fact that you couldn't pull a player's start and end dates with every team from the box score data, because a player may be traded while they're injured, and technically be inactive, so wouldn't even appear in a box score well into their tenure with the team. 

What I think makes sense long term is to create a completely separate table for a player's status, that will contain the player's name/id, the game id, whether or not they were active or inactive, and the reason for being inactive if it's available. In order to capture this, I'd need transaction data, and full NBA game logs, which is captured in the General Game Data which I'll tackle next. In the next post I'll just concentrate on creating the player transaction table.

## General Game Table
The game data I'm looking to extract is found mostly within the same 'game' key that the box score data was embedded inside. However, some of it is also found in the parent key 'analyticsObject'.

The key-value pairs I want to pull from this again are:
- Game Date
- Regular Season/Playoffs
- Home and Away Team
- Final Score
- Stadium Attendance
- National TV Broadcast

Also, there are times where these keys don't exist for all games, so it's important to account for that in cases where they don't. 

In [41]:
game_object = json_object['props']['pageProps']['game']
analytics_object = json_object['props']['pageProps']['analyticsObject']


# GameID
game_id = game_object.get('gameId')

# Date in US Eastern Timezone:
game_date = game_object.get('gameEt')

# Season Year
season_year = analytics_object.get('season')

# Regular Season or Playoffs
season_type = analytics_object.get('seasonType')

# Game Attendance
attendance = game_object.get('attendance')

# Game Sold Out
sellout = game_object.get('sellout')

# Game Duration
duration = game_object.get('duration')

# Game Label
game_label = game_object.get('gameLabel')

# Game Sub-Label (What game of the series was it, game 1, 2... etc)
game_sublabel = game_object.get('gameSubLabel')

# Series Game Number--not sure how this is different from sublabel
series_game_number = game_object.get('seriesGameNumber')

# Series Text (Information about the series if it's a playoff game)
series_text = game_object.get('seriesText')

# The home/away team name, and corresponding NBA ID for the team:
home_team_id = game_object['homeTeam'].get('teamId')
home_team_city = game_object['homeTeam'].get('teamCity')
home_team_name = game_object['homeTeam'].get('teamName')

away_team_id = game_object['awayTeam'].get('teamId')
away_team_city = game_object['awayTeam'].get('teamCity')
away_team_name = game_object['awayTeam'].get('teamName')

# Final Score 
home_team_score = game_object['homeTeam'].get('score')
away_team_score = game_object['awayTeam'].get('score')

# National TV Broadcast
national_broadcasters = game_object['broadcasters'].get('nationalBroadcasters', [])

if national_broadcasters:
    national_tv_broadcast = national_broadcasters[0].get('broadcasterDisplay')
else:
    national_tv_broadcast = None

One note on the above, the broadcast data has a lot of information--it contains TV and radio broadcasts both locally and nationally that played the game--I'm only interested in whether the game was played nationally, which appears to always appear in the 'nationalBroadcasters' key. If it wasn't nationally broadcast, the key still exists, but the list value is just empty. 

There are a lot of variables here that can probably be put into another table, and some that can be joined, like the information regarding the home team city and name. In addition, the game_date and duration should probably be changed to a datetime variable and a integer (for either seconds or minutes) respectively. 

Beyond these minor additional transformations, the next step is to turn this into a pandas dataframe, since I'm going to store this data in a relational database:

In [42]:
data = {
    'game_id' : game_id,
    'game_date' : game_date,
    'season_year' : season_year,
    'season_type' : season_type,
    'attendance' : attendance,
    'sellout' : sellout,
    'duration' : duration,
    'game_label' : game_label,
    'game_sublabel' : game_sublabel,
    'series_text' : series_text,
    'series_game_number' : series_game_number,
    'series_text' : series_text,
    'home_team_id' : home_team_id,
    'home_team_city' : home_team_city,
    'home_team_name' : home_team_name,
    'away_team_id' : home_team_id,
    'away_team_city' : away_team_city,
    'away_team_name' : away_team_name,
    'home_team_score' : home_team_score,
    'away_team_score' : away_team_score,
    'national_tv_broadcast' : national_tv_broadcast
}

df_game_data = pd.DataFrame([data])
df_game_data

,game_id,game_date,season_year,season_type,attendance,sellout,duration,game_label,game_sublabel,series_text,series_game_number,home_team_id,home_team_city,home_team_name,away_team_id,away_team_city,away_team_name,home_team_score,away_team_score,national_tv_broadcast
0,0022101217,2022-04-10T15:30:00Z,2021,Regular Season,18465,0,2:11,None,None,,,1610612766,Charlotte,Hornets,1610612766,Washington,Wizards,124,108,None


We can now iterate through each JSON file pulled from NBA.com and create a corresponding box score table, as well as a table for information on the game itself. 